# Using Pandas and SQL Alchemy to Connect to a Remote Database 
© Explore Data Science Academy

## Introduction  
SQL Alchemy is just another way to access and interact with an SQL Database directly from python. This is useful as Python is a lot more powerful when it comes to manipulating and analysing data. Being able to access one from the other just makes analysis easier and allows us to create pipelines more effectively. 

## Overview  
In this tutorial we will explore a new method to connect to a remorte MS SQL Server using SQL Alchemy, reading a table into pandas, and creating and deleting items using the `execute` command.

## Learning Objectives

In this train, you will learn how to:
* Install SQL Alchemy and pymssql
* Interact with MS SQL Server from Python
* Run SQL queries in Python

## Prerequisites 

* Access to and credentials for an AWS RDS instance
* Tables in your database (This train uses the `ClassList` database).

### Install SQL Alchemy And pymssql

These packages can be found in the Python Package Index and can be  installed using ` pip install` or `conda install` on your administrator command prompt or on a Jupyter notebook (or google colab) cell:


In [0]:
!pip install sqlalchemy
!conda install pymssql
!apt-get -qq install python-pymssql python3-pymssql

In [0]:
import sqlalchemy as sa
import pandas as pd
from sqlalchemy import create_engine 

### Credentials
**NB!** Change the credentials to those specific to you  
**NB!** Make sure you have data in your database to query from - this train assumes you have a database called ClassList restored to your RDS instance.

In [0]:
# Username = master_username
# Password = master_password
# Host = AWS_endpoint
# Port = '1433'
# Database = database_name (not DB Identifier)

The `database_name` is the name you elected to store your database as when you restored it from the S3 bucket. In the video tutorial we used `ClassList`, so that is most likely what yours will be too.

In [0]:
Username = 'admin'
Password = '<auto-generated-by-aws>'
Host = 'db-video-test.cnzbp4ndrpos.eu-west-1.rds.amazonaws.com'
Port = '1433'
Database = 'ClassList'

### Create connection  
You will now use a function of sqlalchemy called `create_engine` to connect to your database. Once again, ensure the credentials above are changed to your own. 

In [0]:
conn = create_engine(
    'mssql+pymssql://' +
    Username + ':' + Password + '@' + Host + ':' + Port + '/' + Database
)

##Create a query
Decide on a query that you'd like to run and save it as a string below. Ensure that is relates to the database you have loaded if different from the football_players database.


In [0]:
sql_query = "Select * from ExamMarks"

### Create a DataFrame from your SQL DB

Using pandas we now take our connection and our query to create a dataframe. Below we have inserted the sql_query and conn variables that we created earlier. We now have a pandas dataframe containing all the data from our query.

In [0]:
df = pd.read_sql_query(sql_query, conn)
df.head()

## Create a new table  

Next, we are going to create our own table and fill it with values. We will need a few additional functions to do this- these are imported below.   

We create a new table called `Table_testing`, and then specifiy the table name, followed by each column with the column name and data type as arguments.   

In [0]:
from sqlalchemy import Table, Column, Integer, Float, String, MetaData
from sqlalchemy.sql import text
from sqlalchemy import insert

In [0]:
m = MetaData()
Table_testing = Table('Table_testing', m, Column('id', Integer, primary_key=True),
                  Column('first_name', String),
                  Column('last_name', String),
                  Column('username', String),
                  Column('email', String),
                  Column('address', String),
                  Column('town', String))

m.create_all(conn)

## Inserting data into your table  
We insert new records using dictionaries. The keys correspond to the column names, and the value is the data you would like to insert into that column. 
Each new record/row requires its own dictionary, as illustrated below. 

In [0]:
conn.execute(Table_testing.insert(), 
  {
    'id' : 51,
    'first_name' : 'Jessica',
    'last_name' : 'Roberts',
    'username' : 'jessrob',
    'email' :'jessie@iol.com',
    'address' : '1234 North Side Avenue',
    'town' : 'Fictville'
    }, # dictionary 1
  {    
    'id' : 52,
    'first_name' : 'Robin',
    'last_name' : 'Johnson',
    'username' : 'robin',
    'email' :'rjohnson@gmail.com',
    'address' : '1236 East Side Blvd',
    'town' : 'Defiance'
   }) # dictionary 2

Let's take a look at our table and see if our insert worked.

In [0]:
sql = 'select * from Table_testing;'
pd.read_sql_query(sql, conn)

If you don't see the data you inserted in the table above, go back and see if there's a step you missed. Otherwise, continue by adding in the records below. 

In [0]:
conn.execute(Table_testing.insert(),  {
    "id":34,
    "first_name": "Jackson", 
    "last_name": "Brown", 
    "username": "jackB", 
    "email":"jbrown@mail.com", 
    "address": "5427 Mandela Drive",
    "town": "Greyston"
        },
        {
    "id": 82,           
    "first_name": "Hermine", 
    "last_name": "Jacobs", 
    "username": "Hjacobs", 
    "email":"hjacobs@mail.com",
    "address": "3572 Penguin Avenue",
    "town": "Sandringham"
        },
        {
    "id":12,            
    "first_name": "Thomas", 
    "last_name": "Tennyson", 
    "username": "TommyT", 
    "email":"tomten@mail.com",
    "address": "1881 Apple Street",
    "town": "Peterbrugh"
        },
        {
    "id":27,           
    "first_name": "Denise", 
    "last_name": "Kramer", 
    "username": "denisekramer", 
    "email":"denisekramer@mail.com",
    "address": "28 Eden Drive",
    "town": "Constantia" })

In [0]:
sql = 'select * from Table_testing;'
pd.read_sql_query(sql, conn)

## Delete a row  
We have decided that one person does not belong in this table. We can execute a command to delete a row from the table based on some condition: in this case, we are deleting anything where the id is 51. Take a look at your table again to ensure the command has been carried out. 

In [0]:
sql = 'DELETE FROM Table_testing WHERE id = 51'
conn.execute(sql)

In [0]:
sql = 'select * from Table_testing;'
pd.read_sql_query(sql, conn)

## Try it yourself

Below we have a template to create multiple tables. We then populate these tables using dictionares as we did previously (add the your own names and values where you see `###` and change the data types as necessary). Try to create and populate your own tables. Just remember to delete them when you are done!

In [0]:
m = MetaData()
tableA = Table('###table name1', m,
                  Column("###column name", String),
                  Column("###column name", Float),
                  Column("###column name", Float),
                  Column("###column name", Float),)
tableB = Table('###table name2', m,
                  Column("###column name", Integer))

tableC= Table('table name3', m,
                  Column("###column name", Integer),
                  Column("###column name", Integer),
                  Column("###column name", Integer),)

m.create_all(conn)

In [0]:
'###table name1' = [
    {
        "###column name":"Soup",
        "###column name": 1.21,
        "###column name": 4.81,
        "###column name": 50
    }
]
 
'###table name2' = [
    {
        "###column name": 89
    }
]
 
'###table name3' = [
    {
        "###column name": 3,
        "###column name": 1,
        "###column name": 5
    }
]
 

In [0]:
r = conn.execute(insert(items), items_list)
r.rowcount
r1= conn.execute(insert(orders), order_list)
r1.rowcount
r2= conn.execute(insert(order_lines), order_line_list)
r2.rowcount
print(r.rowcount,r1.rowcount,r2.rowcount)

## Selecting Records

You can retrieve records by executing the commands below. `fetchall` will retrieve all records from the table specified. If you would just like to see a single record, you can use `fetchone`. Look up a few other commands to retrieve records. 

In [0]:
s = Table_testing.select()
str(s)

In [0]:
r = conn.execute(s)
r.fetchall()

## DROP TABLE
You can also delete a table using sqlalchemy. This is done in the same way you execute other commands. However, you should always be careful when dropping a table or a row as this cannot be undone. 

In [0]:
sql = 'Drop TABLE Table_testing;'
conn.execute(sql)

Drop the remaining tables that you have created - and you're all done.
